# Julia设计模式
* 复用性模式
    * 代表团模式
    * 神圣特性模式
    * parametric type pattern
 

### The parametric type pattern
参数化类型是一个语言用参数具象化数据的一个核心特性。

## 性能模式
Julia编程语言能让我们充分地调用CPU的资源，只要遵守一些规则，我们就能让程序的运行速度更加快速。
以下是性能模式列表：
* Global constant
* Struct of arrays
* Shared arrays
* Memoization
* Barrier function

同样我在网络上找到了一篇讲述如何优化julia运行性能的文章，大家可以去参考一下：https://techytok.com/code-optimisation-in-julia/

那么开始吧~

### 全局常数模式
Julia中，变量要不在全局作用域要不就在局部作用域。
模块顶层中的所有被声明的变量都是全局变量。而函数中声明的变量则是局部的。一般来说，假设一个应用程序调用了外部系统的一个对象，我们不妨称外部对象为handler，那么在程序中，我们会初始化handler为全局变量，因为这样做，外部模块中的所有函数都能够被直接访问。。这是一个便利的因素。

然而，全局变量的声明必然伴随着性能的损失，这会影响到整个程序的性能。所以如何正确地声明全局变量算是门学问。

下面让我们来看看使用全局变量会对性能造成怎样的损失

In [2]:
using BenchmarkTools

variable = 10

function add_using_global_variable(x)
    return x + variable
end

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


add_using_global_variable (generic function with 1 method)

In [3]:
@btime add_using_global_variable(10);

  23.169 ns (0 allocations: 0 bytes)


In [4]:
# 不引用全局变量进行对比
function add_using_function_arg(x, y)
    return x + y
end

add_using_function_arg (generic function with 1 method)

In [5]:
@btime add_using_function_arg(10, 10);

  0.001 ns (0 allocations: 0 bytes)


unbelievable！快了多少倍呀这~

那同学们一定想探索本源，看看为何会快这么多，这么想是对的，因为只有这样做咱们的印象才会深刻。由于Julia是LLVM即时编译的，所以我们可以用内建内省工具**introspection**来看生成的LLVM代码。

In [6]:
@code_llvm add_using_function_arg(10, 10)


;  @ In[4]:3 within `add_using_function_arg'
; Function Attrs: uwtable
define i64 @julia_add_using_function_arg_17964(i64, i64) #0 {
top:
; ┌ @ int.jl:53 within `+'
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


In [7]:
# 可以看到上面的生成的代码只有add一个指令，再来看一下另一个慢的生成的丑陋的代码
# 前方高能,可不要被吓着哦~
@code_llvm add_using_global_variable(10)


;  @ In[2]:6 within `add_using_global_variable'
; Function Attrs: uwtable
define nonnull %jl_value_t addrspace(10)* @julia_add_using_global_variable_17874(i64) #0 {
top:
  %1 = alloca %jl_value_t addrspace(10)*, i32 2
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 4, align 16
  %2 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %2, i8 0, i32 32, i1 false)
  %3 = call %jl_value_t*** inttoptr (i64 1720670528 to %jl_value_t*** ()*)() #6
  %4 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %5 = bitcast %jl_value_t addrspace(10)** %4 to i64*
  store i64 8, i64* %5
  %6 = getelementptr %jl_value_t**, %jl_value_t*** %3, i32 0
  %7 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 1
  %8 = bitcast %jl_value_t addrspace(10)** %7 to %jl_value_t***
  %9 = load %jl_value_t**, %jl_value_t*** %6
  store %jl_value_t** %9, %jl_value_t*** %8
  %10 = bitcast %jl_value_

。。。怎么样，有没有被吓到。
简单来说，为什么会生成那么多代码呢，是因为我们声明的是变量，变量随时能够变化，这意味着编译器必须生成能够处理任意数据类型的代码才能stay on the safe side。
所以还是那句话，灵活性本身必定伴随着巨大的开销。
问题怎么解决？很显然，定义常量，让我们来测试一波吧~

In [8]:
const constant = 10

function add_using_global_constant(x)
    return constant + x
end

add_using_global_constant (generic function with 1 method)

In [9]:
@btime add_using_global_constant(10);

  0.001 ns (0 allocations: 0 bytes)


话又说回来了，我们有时候无可避免地需要定义变量，那怎样做可以提升使用变量时的性能呢？

**答： 标注类型信息**

注意，Julia中全局变量还不支持使用类型声明的方法，因此我们需要在函数定义中去声明。

In [10]:
function add_using_global_variable_typed(x)
    return x + variable::Int
end


add_using_global_variable_typed (generic function with 1 method)

In [11]:
variable = 10

@btime add_using_global_variable_typed(10);

  4.999 ns (0 allocations: 0 bytes)


下面演示一个全局变量的优化方法——“**全局变量占位符（global variable placeholder）**”

Julia的代码会被Julia编译器拿去进行优化，所以我们可以使用常量占位符来存储具体的值。

In [12]:
const semi_constant = Ref(10)

function add_using_global_semi_constant(x)
    return x + semi_constant[]
end

# to fetch the value inside a Red object，we can just use index operator with no argument.
@btime add_using_global_constant(10)

  0.001 ns (0 allocations: 0 bytes)


20

In [13]:
# 这个常量是可以进行更改的，所以可以模拟为变量
semi_constant[] = 20

20

In [14]:
const list = Ref([1])
list

Base.RefValue{Array{Int64,1}}([1])

In [15]:
list[]

1-element Array{Int64,1}:
 1

In [17]:
list[] = [1, 2]

2-element Array{Int64,1}:
 1
 2

### 数组结构模式
如何高效地利用CPU，希望通过下面的例子进行讲解。这个文件所在的目录下有一份纽约市计程车的CSV数据，具体的format可以打开来查看。现在我们希望将CSV里的数据转入Julia变量中，并用struct进行封装。

我们首先使用CSV.jl定义一个读取表的函数：

In [25]:
struct TripPayment
    vendor_id::String
    tpep_pickup_datetime::String
    tpep_dropoff_datetime::String
    passenger_count::Int
    trip_distance::Float64
    fare_amount::Float64
    extra::Float64
    mta_tax::Float64
    tip_amount::Float64
    tolls_amount::Float64
    improvement_surcharge::Float64
    total_amount::Float64
end

In [19]:
# 此处的datarow = 3代表
function read_trip_payment_file(file)
    f = CSV.File(file, datarow = 3)
    records = Vector{TripPayment}(undef, length(f))
    for (i, row) in enumerate(f)
        records[i] = TripPayment(row.VendorID,
            row.tpep_pickup_datetime,
            row.tpep_dropoff_datetime,
            row.passenger_count,
            row.trip_distance,
            row.fare_amount,
            row.extra,
            row.mta_tax,
            row.tip_amount,
            row.tolls_amount,
            row.improvement_surcharge,
            row.total_amount)
    end
    return records
end

read_trip_payment_file (generic function with 1 method)

In [24]:
import CSV
read_trip_payment_file("tripdata.csv")

MethodError: MethodError: Cannot `convert` an object of type Int64 to an object of type String
Closest candidates are:
  convert(::Type{String}, !Matched::FilePathsBase.AbstractPath) at C:\Users\pp\.julia\packages\FilePathsBase\n76Ka\src\path.jl:83
  convert(::Type{String}, !Matched::CategoricalArrays.CategoricalValue) at C:\Users\pp\.julia\packages\CategoricalArrays\0ZAbp\src\value.jl:60
  convert(::Type{String}, !Matched::WeakRefStrings.WeakRefString) at C:\Users\pp\.julia\packages\WeakRefStrings\lqf5B\src\WeakRefStrings.jl:79
  ...